In [5]:
import numpy as np
import helper
import pickle
import datetime
import os.path
import time
import pandas as pd
import sqlite3
import pandas as pd
import re

In [6]:
origin="ATL"
destination="SEA"
mode="domestic"
duration=5
folder_location = "logs/"
exclusion_list = ["2024-05-31.pkl", "2024-05-31_intl.pkl"]
stops = 0

#This gets all the data from all the files that matches origin and destination
# aggDf = helper.getAllFlightData(origin, destination, folder_location, exclusion_list)
# aggDf = helper.getItinDfFromAggDf(origin, destination, aggDf, duration, mode, stops)

In [30]:
def extract_price_and_currency(df):
    # Use regex to extract currency symbol (assumes symbol is at the start)
    df['currency'] = df['price'].str.extract(r'^([^\d]+)')  # non-digit characters at start
    # Remove non-numeric characters and convert to float
    df['price_float'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)
    return df

def query_flights_from_db(db_filename, mode, folder_location, origin=None, destination=None, max_price=None, after_date=None):
    # Connect to the database
    # print(f"{folder_location}{db_filename}_{mode}.db")
    conn = sqlite3.connect(f"{folder_location}{db_filename}_{mode}.db")
    cursor = conn.cursor()

    # Build query with optional conditions
    query = "SELECT * FROM data_table WHERE 1=1"
    params = []

    if origin:
        query += " AND origin = ?"
        params.append(origin)
    if destination:
        query += " AND destination = ?"
        params.append(destination)
    if max_price:
        query += " AND CAST(price AS FLOAT) <= ?"
        params.append(max_price)
    if after_date:
        query += " AND departure_date >= ?"
        params.append(after_date)

    # Execute the query and load into DataFrame
    df = pd.read_sql_query(query, conn, params=params)
    print(f"\nTotal rows fetched into DataFrame: {len(df)}")

    # Close connection
    conn.close()
    return df


def filter_df(
    df,
    max_price=None,
    min_price=None,
    start_date=None,
    end_date=None,
    exact_origin=None,
    exact_destination=None,
    max_stops=None
):
    filtered_df = df.copy()

    if max_price is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['price_float'], errors='coerce') <= max_price]
    if min_price is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['price_float'], errors='coerce') >= min_price]
    if start_date is not None:
        filtered_df = filtered_df[filtered_df['departure_date'] >= start_date]
    if end_date is not None:
        filtered_df = filtered_df[filtered_df['departure_date'] <= end_date]
    if exact_origin:
        filtered_df = filtered_df[filtered_df['origin'] == exact_origin]
    if exact_destination:
        filtered_df = filtered_df[filtered_df['destination'] == exact_destination]
    if max_stops is not None:
        filtered_df = filtered_df[pd.to_numeric(filtered_df['stops'], errors='coerce') <= max_stops]

    print(f"Filtered rows remaining: {len(filtered_df)}")
    return filtered_df

In [8]:

results = query_flights_from_db(
    db_filename="dataDB",
    mode="domestic",
    folder_location="./logs/",
    origin="ATL",
    destination="SEA",
    # max_price=450,
    # after_date="2025-07-01"
)
results = extract_price_and_currency(results)
print(f"\nTotal rows now: {len(results)}")



./logs/dataDB_domestic.db

Total rows fetched into DataFrame: 1684925

Total rows now: 759353


In [37]:
# results = extract_price_and_currency(results)
print(f"\nTotal rows now: {len(results)}")
# print(results.head())
results_nonstop = filter_df(results, max_price=90, max_stops=0)
print(results_nonstop.head())


Total rows now: 759353
Filtered rows remaining: 1111
               id origin destination    name days price       today  \
1344674  31000562    ATL         SEA  Alaska        $79  2024-09-18   
1344675  31000563    ATL         SEA  Alaska        $79  2024-09-18   
1344676  31000564    ATL         SEA   Delta        $79  2024-09-18   
1344677  31000565    ATL         SEA   Delta        $79  2024-09-18   
1344772  31000773    ATL         SEA  Alaska        $79  2024-09-18   

         days_ahead flight_duration           flight_depart  \
1344674          34     5 hr 30 min  7:00 AM on Wed, Oct 23   
1344675          34     5 hr 35 min  6:25 PM on Wed, Oct 23   
1344676          34     5 hr 34 min  7:30 AM on Wed, Oct 23   
1344677          34     5 hr 33 min  7:50 PM on Wed, Oct 23   
1344772          35     5 hr 37 min  3:45 PM on Thu, Oct 24   

                   flight_arrive stops stops_info departure_date currency  \
1344674   9:30 AM on Wed, Oct 23     0                2024-10-2